In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from tpot import TPOTRegressor
from lib.reproduction import major_oxides
from lib import full_flow_dataloader
from lib.norms import Norm1Scaler, Norm3Scaler
import pandas as pd

In [6]:
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)

In [7]:
train_processed, test_processed = full_flow_dataloader.load_full_flow_data()

In [8]:
from sklearn.model_selection import train_test_split
from lib.utils import custom_train_test_split

train_cols = train_processed.columns
test_cols = test_processed.columns
norm = 1

scaler = Norm1Scaler() if norm == 1 else Norm3Scaler()

train = scaler.fit_transform(train_processed)
test = scaler.fit_transform(test_processed)

# turn back into dataframe
train = pd.DataFrame(train, columns=train_cols)
test = pd.DataFrame(test, columns=test_cols)

drop_cols = major_oxides + ["ID", "Sample Name"]

## - VALIDATION -
# split_train, split_val = custom_train_test_split(train, "Sample Name", test_size=0.2, random_state=42)

# X_train = split_train.drop(columns=drop_cols)
# y_train = split_train[major_oxides]
# X_val = split_val.drop(columns=drop_cols)
# y_val = split_val[major_oxides]

# Converting train set - comment out if using validation
X_train = train.drop(columns=drop_cols)
y_train = train[major_oxides]

# Converting test set
X_test = test.drop(columns=drop_cols)
y_test = test[major_oxides]

In [12]:
for oxide in major_oxides:
    tpot.fit(X_train, y_train[oxide])
    print(tpot.score(X_test, y_test[oxide]))

                                                                                  
Generation 1 - Current best internal CV score: -32.77342155019121
                                                                                  
Generation 2 - Current best internal CV score: -31.79131536902286
                                                                                  
Generation 3 - Current best internal CV score: -29.948997668204747
                                                                                 
Generation 4 - Current best internal CV score: -29.948997668204747
                                                                                 
Generation 5 - Current best internal CV score: -29.655941571927684
                                                             
Best pipeline: ExtraTreesRegressor(MaxAbsScaler(RidgeCV(MinMaxScaler(input_matrix))), bootstrap=True, max_features=0.35000000000000003, min_samples_leaf=19, min_samples_split=19, n_estimators=

ValueError: Error: Input data is not in a valid format. Please confirm that the input data is scikit-learn compatible. For example, the features must be a 2-D array and target labels must be a 1-D array.

In [13]:
tpot.export(f"tpot_{major_oxides[0]}_pipeline.py")

In [18]:
# calculate rmse score for tpot
from sklearn.metrics import mean_squared_error
from math import sqrt

y_pred = tpot.predict(X_test)
rmse = sqrt(mean_squared_error(y_test[major_oxides[0]], y_pred))
print(rmse)

3.9426960700136946
